<span style="color: gold;"> 벡터 DB = 벡터 스토어

<span style="font-size:13px;">

- 텍스트·이미지 등을 벡터로 저장하고, 유사도 기반으로 검색하는 DB를 통칭하는 말
- 데이터베이스는 쿼리가 정확하게 일치해야지 결과를 출력한다 ( 대한민국 =! 한국 -> 다르게 인식)
- 벡터DB는 단어의 의미와 맥락을 기반으로 데이터를 찾는다
- 벡터 단위의 유사도를 보고 출력해주는 것 중에 하나가  <span style="color: yellow">크로마DB(ChromaDB)

---

<span style="color: gold;"> 1. 기본 ChromaDB 임베딩 사용

In [ ]:
# 임베딩을 따로 정의안하고 기본으로 사용한 chromadb  

import chromadb
# 1. 클라이언트 생성 (메모리에만 저장)
client = chromadb.Client() #--> 인메모리 방식 / 객체를 만들었다라고 생각

# 2. 컬렉션 생성  (데이터베이스에 스키마, 테이블.. 같은걸 만들었다 생각)
collection = client.get_or_create_collection('my_collection')  # get_or_create_collection 있으면 가져오고 없으면 만들어라/ 컬렉션 이름운 유니크해야 함

# 3. 데이터를 추가
import uuid
collection.add(
    documents = ['화면을 보여주는 모니터', '글자를 입력하는 키보드', '커서를 움직이는 마우스'], # 저장할 텍스트(임베딩될 문장)
    metadatas = [{'category' : 'display'}, {'category':'input'}, {'category': 'input'}], # 문장에 붙일추가 정보 (옵션)
    ids = [str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4())]       # 필수, 각 문서의 고유 식별자. ids는 문자열만 허용해서 str()씌워줌
)

# 4. 질문하기 (의미기반)
results = collection.query(
    query_texts=['숫자를 입력하는 기계'],
    n_results= 1 #결과 하나만
)

results

# 결과가 좋지 않음. chromadb가 한국어가 별로임 # default languge 를 사용해서.
    # query_texts=['뭔가를 보여주는 기계'], ---> 출력  'documents': [['커서를 움직이는 마우스']],
    # query_texts=['숫자를 입력하는 기계'] ---> 출력 'documents': [['글자를 입력하는 키보드']],

C:\Users\SAMSUNG\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:20<00:00, 4.03MiB/s]


{'ids': [['732aeb38-f83c-49b6-bebc-8d7112ff5035']],
 'embeddings': None,
 'documents': [['글자를 입력하는 키보드']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'category': 'input'}]],
 'distances': [[0.6260867714881897]]}

<span style="color: gold;">  2. OpenAI을 사용한 임베딩

In [ ]:

import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os
from dotenv import load_dotenv
load_dotenv()

# 1. 클라이언트 생성 (메모리에만 저장)

client = chromadb.Client() #--> 인메모리 방식 / 객체를 만들었다라고 생각
embed_fn = OpenAIEmbeddingFunction( 
            api_key= os.environ['OPENAI_API_KEY'],
            model_name="text-embedding-3-small"
        )
    

# 2. 컬렉션 생성  (데이터베이스에 스키마, 테이블.. 같은걸 만들었다 생각)
collection = client.get_or_create_collection('my_collection2', embedding_function=embed_fn)  




# 3. 데이터를 추가
import uuid
collection.add(
    documents = ['화면을 보여주는 모니터', '글자를 입력하는 키보드', '커서를 움직이는 마우스'], # 벡터가 될 문장
    metadatas = [{'category' : 'display'}, {'category':'input'}, {'category': 'input'}], # 문장들과 같이 사용할 .. ? 인덱스화 됐다  #필수아니고, 옵션.
    ids = [str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4())]       # 필수, 각각의 고유번호. ids는 str만 허용해서 str()씌워줌
)

# 4. 질문하기 (의미기반)
results = collection.query(
    query_texts=['뭔가를 보여주는 기계'],
    n_results= 1 #결과 하나만
)

results

# 결과 나아짐
    # query_texts=['뭔가를 보여주는 기계'], ---> 출력   'documents': [['화면을 보여주는 모니터']]

{'ids': [['018f78d0-5892-494b-a282-416ef56e979b']],
 'embeddings': None,
 'documents': [['화면을 보여주는 모니터']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'category': 'display'}]],
 'distances': [[0.5194305777549744]]}

<span style="color: gold;">  3. 허깅페이스의 모델을 사용한 임베딩

In [5]:
### Hugging Face 의  model_name= "paraphrase-multilingual-MiniLM-L12-v2" 임베딩 사용



import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os
from dotenv import load_dotenv
load_dotenv()

# 1. 클라이언트 생성 (메모리에만 저장)

client = chromadb.Client() 

from chromadb.utils import embedding_functions
embed_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name= "paraphrase-multilingual-MiniLM-L12-v2"
)
    

# 2. 컬렉션 생성  (데이터베이스에 스키마, 테이블.. 같은걸 만들었다 생각)
collection = client.get_or_create_collection('my_collection3', embedding_function=embed_fn)  




# 3. 데이터를 추가
import uuid
collection.add(
    documents = ['화면을 보여주는 모니터', '글자를 입력하는 키보드', '커서를 움직이는 마우스'], # 벡터가 될 문장
    metadatas = [{'category' : 'display'}, {'category':'input'}, {'category': 'input'}], # 문장들과 같이 사용할 .. ? 인덱스화 됐다  #필수아니고, 옵션.
    ids = [str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4())]       # 필수, 각각의 고유번호. ids는 str만 허용해서 str()씌워줌
)

# 4. 질문하기 (의미기반)
results = collection.query(
    query_texts=['뭔가를 보여주는 기계'],
    n_results= 1 #결과 하나만
)

results

# 결과 좋지 않음
    # query_texts=['뭔가를 보여주는 기계'], ---> 출력  'documents': [['커서를 움직이는 마우스']],

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\SAMSUNG\miniconda3\envs\llm_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SAMSUNG\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'ids': [['14f6681e-4595-42fc-8e56-0a1b0a105ef2']],
 'embeddings': None,
 'documents': [['커서를 움직이는 마우스']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'category': 'input'}]],
 'distances': [[0.2587946653366089]]}